# Test Utils

> Utilities for writting tests.

In [1]:
# | default_exp testing.utils

In [2]:
# |export
# standard
import logging
import os
import shutil
from pathlib import Path
from typing import List, Tuple, Optional
import re
import operator

# 3rd party
from execnb.nbio import new_nb, write_nb, mk_cell, read_nb
from plum import Val

# ours
from nbmodular.core.utils import cd_root

## Notebook examples

> Example notebooks used for testing

### Simple example 1

In [3]:
# | export
nb1 = """
[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)
"""

### Simple example 2

In [4]:
nb2 = """
[markdown]
# Second notebook

[code]
%%function bye
print ('bye')

[markdown]
%%function two_plus_two --test
a=2+2
print (a)
"""

### Mixed Cells Example

In [ ]:
# | export
mixed_nb1 = """
[code]
%%function
def first():
    pass

[markdown]
comment
    
[code]
%%function --test
def second ():
    pass
"""

## Python module examples

> Example python modules used here

### Example 1

In [ ]:
# | export
py1 = """
def hello ():
    print ('hello')

def one_plus_one ():
    a=1+1
    print (a)
"""

### Simple example 2

In [ ]:
py2 = """
def bye ():
    print ('bye')

def two_plus_two ():
    a=2+2
    print (a)
"""

## Notebook structure

> Utilities for building a dictionary with notebook structure. Useful for testing purposes.

### convert_nested_nb_cells_to_dicts

In [5]:
# | export
def convert_nested_nb_cells_to_dicts(dict_like_with_nbcells: dict) -> dict:
    """Convert nested NbCells to dicts.

    Parameters
    ----------
    dict_like_with_nbcells : dict
        dict-like object with embedded NbCell cells

    Returns
    -------
    dict
        dict object without embedded NbCell cells
    """
    new_dict = {k: v for k, v in dict_like_with_nbcells.items()}
    new_dict["cells"] = [dict(**cell) for cell in new_dict["cells"]]
    return new_dict

### parse_nb_sections

In [6]:
# | export
def parse_nb_sections(nb):
    # Define the regex pattern to match sections
    pattern = "\[(markdown|code)\](.*?)((?=\[markdown\])|(?=\[code\])|$)"

    # Find all matches using re.findall which returns a list of tuples
    matches = re.findall(pattern, nb, re.DOTALL)

    # Transform the matches to the required format
    result = [(match[0], match[1].strip()) for match in matches]

    return result

#### Example usage

In [7]:
nb_text = parse_nb_sections(nb1)
assert nb_text == [
    ("markdown", "# First notebook"),
    ("code", "%%function hello\nprint ('hello')"),
    ("code", "%%function one_plus_one --test\na=1+1\nprint (a)"),
]

### text2nb

In [8]:
# | export
def text2nb(nb: str):
    cells = [
        mk_cell(text, cell_type=cell_type) for cell_type, text in parse_nb_sections(nb)
    ]
    return new_nb(cells)

#### Example usage

In [9]:
nb_text = text2nb(nb1)

#### checks

In [10]:
expected = {
    "cells": [
        {
            "cell_type": "markdown",
            "source": "# First notebook",
            "directives_": {},
            "metadata": {},
            "idx_": 0,
        },
        {
            "cell_type": "code",
            "source": "%%function hello\nprint ('hello')",
            "directives_": {},
            "metadata": {},
            "idx_": 1,
        },
        {
            "cell_type": "code",
            "source": "%%function one_plus_one --test\na=1+1\nprint (a)",
            "directives_": {},
            "metadata": {},
            "idx_": 2,
        },
    ],
    "metadata": {},
    "nbformat": 4,
    "nbformat_minor": 5,
}
actual = convert_nested_nb_cells_to_dicts(
    nb_text
)  # just for comparison purposes, we convert nested NbCells to dicts
assert actual == expected

### texts2nbs

In [11]:
# | export
def texts2nbs(nbs: List[str] | str) -> List[dict]:
    if not isinstance(nbs, list):
        nbs = [nbs]
    return [text2nb(nb) for nb in nbs]

### nb2text

In [12]:
# | export
def nb2text(nb: dict) -> str:
    return "\n\n".join(
        [f"[{cell['cell_type']}]\n{cell['source']}" for cell in nb["cells"]]
    )


def nbs2text(nbs: List[dict]) -> List[str]:
    return [nb2text(nb) for nb in (nbs if isinstance(nbs, list) else [nbs])]

#### Usage example

In [13]:
nb_text = text2nb(nb1)
nb_text = nb2text(nb_text)
assert (
    nb_text
    == """[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)"""
)

In [14]:
nb_text

"[markdown]\n# First notebook\n\n[code]\n%%function hello\nprint ('hello')\n\n[code]\n%%function one_plus_one --test\na=1+1\nprint (a)"

In [15]:
"""[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)"""

"[markdown]\n# First notebook\n\n[code]\n%%function hello\nprint ('hello')\n\n[code]\n%%function one_plus_one --test\na=1+1\nprint (a)"

### printnb

In [16]:
# | export
def printnb(
    nb_text: str | dict | List[str] | List[dict], no_newlines: bool = False, titles=None
) -> None:
    if isinstance(nb_text, list):
        assert titles is None or len(titles) == len(nb_text)
        titles = (
            ["\n"] * len(nb_text)
            if titles is None
            else ["\n" + title for title in titles]
        )
        for nb_text, title in zip(nb_text, titles):
            print(title)
            print(f"{'-'*50}")
            printnb(nb_text, no_newlines=no_newlines)
    else:
        if isinstance(nb_text, dict):
            nb_text = nb2text(nb_text)
        print(f'''"""{nb_text}"""''' if no_newlines else f'''"""\n{nb_text}\n"""''')

#### Usage example

In [17]:
print("-" * 50)
print("with new lines at beginning and end:")
printnb(nb1)
print()
print("-" * 50)
print("without new lines at beginning and end:")
printnb(nb1, no_newlines=True)

--------------------------------------------------
with new lines at beginning and end:
"""

[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)

"""

--------------------------------------------------
without new lines at beginning and end:
"""
[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)
"""


In [18]:
printnb([nb1, nb1], titles=["Number 1", "Number 2"], no_newlines=True)


Number 1
--------------------------------------------------
"""
[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)
"""

Number 2
--------------------------------------------------
"""
[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)
"""


## Check utilities

### strip_nb

In [19]:
# | export
def strip_nb(nb: str) -> str:
    return nb2text(text2nb(nb))

### check_test_repo_content

In [26]:
# | export
def check_test_repo_content(
    current_root: str,
    new_root: str,
    nb_folder: str,
    nb_paths: List[str],  # type: ignore
    nbs: Optional[List[str]] = None,
    show_content: bool = False,
    clean: bool = False,
    output: bool = False,
    keep_cwd: bool = False,
):
    """
    Check the content of a test repository.

    Parameters
    ----------
    current_root : str
        The current root directory.
    new_root : str
        The new root directory.
    nb_folder : str
        The folder containing the notebooks.
    nb_paths : List[str]
        The list of notebook paths.
    nbs : Optional[List[str]], optional
        The list of expected notebook contents, by default None.
    show_content : bool, optional
        Whether to print the notebook contents, by default False.
    clean : bool, optional
        Whether to remove the new root directory, by default False.
    output : bool, optional
        Whether to return the notebook contents and paths, by default False.
    keep_cwd : bool, optional
        Whether to keep the current working directory unchanged, by default False.

    Returns
    -------
    Tuple[List[str], List[str]] or None
        If `output` is True, returns a tuple containing the notebook contents and paths.
        Otherwise, returns None.
    """

    assert Path(current_root).name == "nbmodular"
    new_wd = os.getcwd()

    assert Path(new_wd).resolve() == Path(f"{current_root}/{new_root}").resolve()
    os.chdir(current_root)
    assert (Path(new_root) / "settings.ini").exists()
    nb_paths: List[Path] = [
        Path(f"{new_root}/{nb_folder}/{nb_path}") for nb_path in nb_paths
    ]

    all_files = []
    for nb_path in nb_paths:
        all_files += os.listdir(nb_path.parent)
    assert all_files == [nb_path_i.name for nb_path_i in nb_paths]

    nbs_in_disk = []
    for nb_path in nb_paths:
        assert nb_path.exists()
        nbs_in_disk.append(read_nb(nb_path))

    if nbs is not None:
        assert [strip_nb(nb2text(nb)) for nb in nbs_in_disk] == [
            strip_nb(nb) for nb in nbs
        ]
    if show_content:
        printnb(nbs_in_disk, no_newlines=True)
    if clean:
        shutil.rmtree(new_root)
    if keep_cwd:
        if clean:
            raise ValueError("keep_cwd can't be True if clean is True")
        os.chdir(new_root)
    if output:
        return nbs_in_disk, nb_paths

##### Example usage

See checks after example usage for `create_test_content`

### derive_nb_paths_and_py_paths

In [ ]:
# | export
def derive_nb_paths_and_py_paths(
    nb_paths: List[str],
    new_root: str | Path,
    nbm_folder: str = "nbm",
    tmp_folder: str = ".nbs",
    nbs_folder: str = "nbs",
    lib_folder: str = "nbmodular",
):
    all_nb_paths = []
    for nb_path in nb_paths:
        all_nb_paths.append(Path(new_root) / nbm_folder / nb_path)
        all_nb_paths.append(Path(new_root) / nbs_folder / nb_path)
        tmp_nb = Path(new_root) / tmp_folder / nb_path
        all_nb_paths.append(tmp_nb)
        tmp_test_nb = tmp_nb.parent / f"test_{tmp_nb.name}"
        all_nb_paths.append(tmp_test_nb)
    py_paths = []
    for nb_path in nb_paths:
        original_nb_path = Path(nb_path)
        py_paths.append(
            Path(new_root)
            / lib_folder
            / original_nb_path.parent
            / f"{original_nb_path.stem}.py"
        )
        py_paths.append(
            Path(new_root)
            / lib_folder
            / "tests"
            / original_nb_path.parent
            / f"test_{original_nb_path.stem}.py"
        )
    return all_nb_paths, py_paths

#### Example usage

In [ ]:
nb_paths, py_paths = derive_nb_paths_and_py_paths(
    nb_paths=["folder_A/nb_A.ipynb", "folder_B/nb_B.ipynb"], new_root="tmp_repo"
)
assert nb_paths == [
    Path("tmp_repo/nbm/folder_A/nb_A.ipynb"),
    Path("tmp_repo/nbs/folder_A/nb_A.ipynb"),
    Path("tmp_repo/.nbs/folder_A/nb_A.ipynb"),
    Path("tmp_repo/.nbs/folder_A/test_nb_A.ipynb"),
    Path("tmp_repo/nbm/folder_B/nb_B.ipynb"),
    Path("tmp_repo/nbs/folder_B/nb_B.ipynb"),
    Path("tmp_repo/.nbs/folder_B/nb_B.ipynb"),
    Path("tmp_repo/.nbs/folder_B/test_nb_B.ipynb"),
]
assert py_paths == [
    Path("tmp_repo/nbmodular/folder_A/nb_A.py"),
    Path("tmp_repo/nbmodular/tests/folder_A/test_nb_A.py"),
    Path("tmp_repo/nbmodular/folder_B/nb_B.py"),
    Path("tmp_repo/nbmodular/tests/folder_B/test_nb_B.py"),
]

### read_nbs

In [ ]:
# | export
def read_nbs(paths: List[str], as_text: bool = True) -> List[str] | List[dict]:
    """
    Read notebooks from disk.

    Parameters:
        paths (List[str]): A list of paths to the notebooks.
        as_text (bool, optional): If True, the notebooks will be returned as text.
            If False, the notebooks will be returned as dictionaries.
            Defaults to True.

    Returns:
        List[str] | List[dict]: A list of notebooks. If `as_text` is True, the notebooks
            will be returned as text. If `as_text` is False, the notebooks will be
            returned as dictionaries.
    """
    nbs_in_disk = []
    for path in paths:
        # Check that file exists. useful for being called inside a test utility
        # to see where it fails.
        assert os.path.exists(path)
        nbs_in_disk.append(read_nb(path))

    return [strip_nb(nb2text(nb)) for nb in nbs_in_disk] if as_text else nbs_in_disk

### write_nbs

In [ ]:
def write_nbs(nbs: List[str], nb_paths: List[str]) -> None:
    for nb, path in zip(nbs, nb_paths):
        write_nb(text2nb(nb), path)

### compare_nbs

In [ ]:
def compare_nb(nb1: str, nb2: str) -> bool:
    return strip_nb(nb1) == strip_nb(nb2)


def compare_nbs(nbs1: List[str], nbs2: List[str]) -> bool:
    return all(map(compare_nb, nbs1, nbs2))

#### Example usage

In [ ]:
nbs = [nb1, nb2]
nb_paths = ["first.ipynb", "second.ipynb"]
write_nbs(nbs, nb_paths)
nbs_in_disk = read_nbs(nb_paths)
assert compare_nbs(nbs_in_disk, nbs)
for nb_path in nb_paths:
    Path(nb_path).unlink()

### read_pymodules

In [ ]:
def read_text_files(paths: List[str]) -> List[str]:
    """
    Read the contents of Python modules from the given paths.

    Parameters
    ----------
    paths : List[str]
        A list of file paths to Python modules.

    Returns
    -------
    List[str]
        A list of strings containing the contents of the Python modules.

    Raises
    ------
    AssertionError
        If a file path does not exist.

    """
    text_files = []
    for path in paths:
        # Check that file exists. useful for being called inside a test utility
        # to see where it fails.
        assert os.path.exists(path)
        with open(path, "rt") as file:
            text_files.append(file.read())
    return text_files

### write_text_files

In [ ]:
def write_text_files(texts: List[str], paths: List[str]) -> None:
    for text, path in zip(texts, paths):
        with open(path, "wt") as file:
            file.write(text)

### compare_texts

In [ ]:
def compare_texts(texts1: List[str], texts2: List[str]) -> bool:
    return all(map(operator.eq, texts1, texts2))

#### Example usage

In [ ]:
texts = [py1, py2]
paths = ["first.py", "second.py"]
write_text_files(texts, paths)
texts_in_disk = read_text_files(paths)
assert compare_texts(texts_in_disk, texts)

# clean
for path in paths:
    Path(path).unlink()

### read_and_print

In [ ]:
def read_and_print(
    paths: List[str], file_type: str, print_as_list: bool = False
) -> None:
    if file_type == "notebook":
        files = read_nbs(paths)
    elif file_type == "python":
        files = read_text_files(paths)
    else:
        raise ValueError(f"file_type {file_type} not recognized")
    if print_as_list:
        print(f"[")
    for file in files:
        if not print_as_list:
            print(f"{'-'*50}")
        print('"""')
        print(file)
        print('"""')
        if print_as_list:
            print(",")

## Create tests

### create_and_cd_to_new_root_folder

In [21]:
# | export
def create_and_cd_to_new_root_folder(
    root_folder: str | Path,
    config_path: str | Path = "settings.ini",
) -> Path:
    """Creates `root_folder`, cds to it, and makes it act as *new root* (see below).

    In order to make it the new root, it copies the file `settings.ini`, which
    allows cd_root () find it and cd to it, and also allows some modules to load
    the global root's config from it.

    It assumes that

    Parameters
    ----------
    root_folder : str or Path
        Path to new root.
    config_path : str or Path, optional
        path to roo'ts config file, by default "settings.ini"

    Returns
    -------
    Path
        Absolute path to root_folder, as Path object.
    """
    config_path = Path(config_path)
    root_folder = Path(root_folder).absolute()
    root_folder.mkdir(parents=True, exist_ok=True)
    shutil.copyfile(config_path, root_folder / config_path.name)
    os.chdir(root_folder)

    return root_folder

### create_test_content

In [22]:
# | export
def create_test_content(
    nbs: List[str] | str,
    nb_paths: Optional[List[str] | List[Path] | str | Path] = None,
    nb_folder: str = "nbm",
    new_root: str = "new_test",
    config_path: str = "settings.ini",
) -> Tuple[str, List[str]]:
    """
    Create test content for notebooks.

    Parameters:
        nbs (List[str] | str): List of notebook texts or a single notebook text.
        nb_paths (Optional[List[str] | List[Path] | str | Path]): List of notebook paths or a single notebook path.
            If None, automatically generates notebook paths based on the number of notebooks.
        nb_folder (str): Name of the notebook folder.
        new_root (str): Name of the new root folder.
        config_path (str): Path to the configuration file.

    Returns:
        Tuple[str, List[str]]: A tuple containing the current root folder path and the list of notebook paths.
    """

    # we start from the root folder of our repo
    cd_root()
    current_root = os.getcwd()

    # Convert input texts into corresponding dicts with notebook structure
    nbs = texts2nbs(nbs)

    # Generate list of nb_paths if None
    if nb_paths is None:
        nb_paths = [f"f{idx}" for idx in range(len(nbs))]
    else:
        if not isinstance(nb_paths, list):
            nb_paths = [nb_paths]
        if len(nb_paths) != len(nbs):
            raise ValueError("nb_paths must have same number of items as nbs")

    for nb, nb_path in zip(nbs, nb_paths):
        full_nb_path = Path(new_root) / nb_folder / nb_path
        full_nb_path.parent.mkdir(parents=True, exist_ok=True)
        write_nb(nb, full_nb_path)

    # Copy settings.ini in new root folder, so that this file
    # can be read later on by our export / import functions.
    # Also, cd to new root folder.
    _ = create_and_cd_to_new_root_folder(new_root, config_path)

    return current_root, nb_paths

#### Example usage

In [29]:
# just for checking later
cwd = os.getcwd()

# usage
new_root = "test_create_test_content"
nb_folder = "nbm"
current_root, nb_paths = create_test_content(
    nbs=[nb1, nb2],
    nb_paths=["first_folder/first.ipynb", "second_folder/second.ipynb"],
    nb_folder=nb_folder,
    new_root=new_root,
)

#### checks and cleaning

In [30]:
check_test_repo_content(
    current_root,
    new_root,
    nb_folder,
    nb_paths,
    nbs=[nb1, nb2],
    show_content=True,
    clean=True,
)



--------------------------------------------------
"""[markdown]
# First notebook

[code]
%%function hello
print ('hello')

[code]
%%function one_plus_one --test
a=1+1
print (a)"""


--------------------------------------------------
"""[markdown]
# Second notebook

[code]
%%function bye
print ('bye')

[markdown]
%%function two_plus_two --test
a=2+2
print (a)"""
